In [10]:
import pandas as pd
import numpy as np
df_new= pd.read_csv('/home/manpreet/mps/cc/aclImdb_v1/aclImdb/movie_reviews.csv')
df_new.head()


1    25000
0    25000
Name: sentiment, dtype: int64

In [11]:

df_new.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [12]:
from sklearn.utils import shuffle
df_new = shuffle(df_new)
df_new.head()

import re
def preprocessor(text):
    
   text = re.sub('<[^>]*>','',text)
   emoticons = re.findall('[:;=](?:-)', text)
   text = (re.sub('[\W]+',' ',text.lower())+ ' '.join(emoticons).replace('-',''))
   
   return text

In [13]:
preprocessor("</a>This is# a ;test ::-)!</a>")

'this is a test :'

In [15]:
df_new['review'] = df_new['review'].apply(preprocessor)
df_new.head()

,review,sentiment
35023,please do not waste any money on this movie it...,0
21380,david duchovny plays the lead role in this fil...,1
26073,please don t waste your money on this sorry ex...,0
33290,at first sight who s singing over there just s...,1
34647,i ve avoided seeing this film for some time bu...,1


In [16]:
df_new.index  =range(50000)
df_new.head()

,review,sentiment
0,please do not waste any money on this movie it...,0
1,david duchovny plays the lead role in this fil...,1
2,please don t waste your money on this sorry ex...,0
3,at first sight who s singing over there just s...,1
4,i ve avoided seeing this film for some time bu...,1


In [17]:
X_train = df_new.loc[:2500,'review'].values
y_train = df_new.loc[:2500,'sentiment'].values
X_test = df_new.loc[2500:,'review'].values
y_test = df_new.loc[2500:,'sentiment'].values

In [18]:
print(np.bincount(y_test))
print(np.unique(y_test))

[23757 23743]
[0 1]


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression   #for predection
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import GridSearchCV #hyper-path
from sklearn.model_selection import cross_val_score

In [25]:
tfidf = TfidfVectorizer(stop_words='english')

param_grid = {'clf__C':[1.0,10.0,100.0]}
X_train.ndim

1

In [26]:
Ir_tfidf = Pipeline([('vect',tfidf),('clf',LogisticRegression())])

gs_Ir_tfidf = GridSearchCV(Ir_tfidf,param_grid,scoring='accuracy')

In [31]:
gs_Ir_tfidf.fit(X_train,y_train)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__C': [1.0, 10.0, 100.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=0)

In [34]:
print('Best parameter set: %s ' % gs_Ir_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_Ir_tfidf.best_score_)


Best parameter set: {'clf__C': 10.0} 
CV Accuracy: 0.842


In [36]:
clf = gs_Ir_tfidf.best_estimator_

In [37]:
clf.score(X_test,y_test)

0.8530947368421052

In [38]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
nb = Pipeline([('vect', tfidf),('clf', MultinomialNB())])

In [40]:
nb.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [41]:
nb.score(X_train,y_train)

0.9812075169932027

In [42]:
nb.score(X_test,y_test)

0.8444842105263158

In [43]:
cv = TfidfVectorizer(stop_words='english')

In [45]:
new_data = cv.fit_transform(X_train)
new_test = cv.transform(X_test)

In [46]:
new_data.shape

In [47]:
nb = MultinomialNB()
nb.fit(new_data,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [48]:
nb.score(new_test,y_test)

0.8444842105263158

In [49]:
nb.score(new_data,y_train)

0.9812075169932027